In [ ]:
# clean last run result

!('./clean.sh')

In [ ]:
from thunder_ase.fireball import Fireball
import numpy as np
import ase

### Benzene Structure

In [ ]:
from ase.build import molecule

In [ ]:
atoms = atoms = molecule('C6H6')

### parameters for fireball

In [ ]:
# set Fdata dir
Fdata_path = '../../data/Fdata-McWEDA-0.15-3SN.Hs3.75.Cs4.00p4.45.Os3.35p3.80-3SNP.Fes5.30p5.30d4.80'

In [ ]:
kwargs = {'iwriteout_charges': 1,  # Writing out the charges.
          'iwriteout_cdcoeffs': 1,  # Writing out orbital info.
          'taurelax': 5.0,
          'efermi_T': 200.0,
          'ifix_CHARGES': 0,
          'max_scf_iterations_set': 100,
          'scf_tolerance_set': 0.00000001,
          'beta_set': 0.04,
          }

### Run Fireball

In [ ]:
calc = Fireball(command='lightning.3.x', 
                Fdata_path=Fdata_path,
                **kwargs)
atoms.set_calculator(calc)

In [ ]:
# Not neccessory but recommended
e0 = atoms.get_potential_energy()
efermi = atoms.calc.get_fermi_level()

print("The energy is {:.3f} eV.".format(e0))
print("The Fermi Level is {:.3f} eV.".format(efermi))

In [ ]:
# write mwfn file

calc.write_mwfn()

### Run Multiwfn in terminal and load .mwfn file